### Keras CNN으로 폐렴 X-Ray 구분하기

1. 패키지 수입 및 하이퍼 파라미터 설정

In [1]:
# 패키지 수입
import os
import random
import numpy as np
import matplotlib.pyplot as plt

from time import time
from sklearn.metrics import confusion_matrix, f1_score

from keras.models import Model
from keras.layers import Conv2D, SeparableConv2D
from keras.layers import Input, Flatten
from keras.layers import BatchNormalization, MaxPool2D # 매 층마다 z-정규화
from keras.layers import Dense, Dropout
from keras.preprocessing.image import load_img


In [2]:
# 하이퍼 파라미터 설정
MY_EPOCH = 200
MY_BATCH = 100
MY_RES = 180
MY_SHAPE = (MY_RES, MY_RES, 3) # 칼라 이미지

# 모드 설정
DATA_MODE = 1
TRAIN_MODE = 1
MY_PATH = '/content/drive/MyDrive/Colab Notebooks/data/chest'

# keras.io에서 pneumonia Classfication 참고하기

2. 데이터 처리

In [3]:
# 변수 정의
# T_tot : train total, 학습용 이미지 개수
# V_tot : validation total, 평가용 이미지 개수
# N_tot : normal total, 정상 이미지 수
# P_tot : pnuemonia total, 폐렴 이미지 수
# N_path : normal image path, 정상 이미지 경로
# P_path : pneumonia image path, 폐렴 이미지 경로

# 전역 변수 설정
T_tot = V_tot = N_tot = P_tot = 0
N_path = P_path = []

X_train = np.zeros((0)) # 아무것도 들어있지 않은 0차원 np
Y_train = np.zeros((0)) 
X_test = np.zeros((0)) 
Y_test = np.zeros((0))

In [4]:
# 학습용 입력 이미지 경로 처리
def train_path():
    global T_tot, N_tot, P_tot, N,path, P_path

    # 정상 이미지 처리
    dir = os.path.join(MY_PATH, 'train/NORMAL') # 경로 두개를 합침
    print(dir)
    
    # 이미지 파일 경로
    N_path = []
    for f in os.listdir(dir):
        N_path.append(os.path.join(dir, f)) # dir과 파일 이름을 합침 -> 파일의 총 경로가 됨
    N_tot = len(N_path)
    print('학습용 정상 이미지 수 :', N_tot)
    
    # 폐렴 이미지 처리
    dir = os.path.join(MY_PATH, 'train/PNEUMONIA')
    print(dir)

    # 이미지 파일 경로
    P_path = []
    for f in os.listdir(dir):
        P_path.append(os.path.join(dir, f)) # dir과 파일 이름을 합침 -> 파일의 총 경로가 됨
    P_tot = len(P_path)
    print('학습용 폐렴 이미지 수 :', P_tot)

    # 총 학습용 이미지 수
    T_tot = N_tot + P_tot
    print('학습용 이미지 수 :', T_tot)

In [5]:
# 학습용 이미지 데이터 처리 (화상도 통일)
# 정상 라벨은 0, 폐렴은 1
def train_resize():
    global T_tot, N_tot, P_tot, N,path, P_path
    global X_train, Y_train

    print('정상 이미지 처리 시작')
    begin = time()

    # 초기화
    X_train = np.zeros((T_tot, MY_RES, MY_RES, 3)) # 데이터 수, 이미지 화소수, 칼라
    Y_train = np.zeros((T_tot,)) # 1차원 벡터

    for i, path in enumerate(N_path):    
        #tmp = load_img(path)
        #print(np.array(tmp).shape)
        #plt.imshow(tmp)
        #plt.show()

        # 이미지 크기 조정
        img = load_img(path,
                       target_size=(MY_RES, MY_RES))
        
        #plt.imshow(tmp)
        #plt.show()

        X_train[i] = img
        Y_train[i] = 0 # 라벨 정보
        #print(X_train[i].shape)

    end = time()
    print('정상 이미지 처리 소요 시간 : {:.2f}'.format(end - begin))

    print('폐렴 이미지 처리 시작')
    begin = time()

    for i, path in enumerate(P_path):
        img = load_img(path, 
                       target_size=(MY_RES, MY_RES))
        X_train[i + N_tot] = img 
        Y_train[i + N_tot] = 1 # 폐렴은 1

    end = time()
    print('폐렴 이미지 처리 소요 시간 : {:.2f}'.format(end - begin))

    # [0, 255] 사이의 8-비트 칼라 정보를 [0,1]로 스케일링(정규화)
    #print(X_train[0])
    X_train = X_train / 255.0

In [6]:
# 평가용 데이터 경로 처리
def test_path():
    global V_tot, N_tot, P_tot, N,path, P_path

    # 정상 이미지 처리
    dir = os.path.join(MY_PATH, 'test/NORMAL') # 경로 두개를 합침
    print(dir)
    
    # 이미지 파일 경로
    N_path = []
    for f in os.listdir(dir):
        N_path.append(os.path.join(dir, f)) # dir과 파일 이름을 합침 -> 파일의 총 경로가 됨
    N_tot = len(N_path)
    print('평가용 정상 이미지 수 :', N_tot)
    
    # 폐렴 이미지 처리
    dir = os.path.join(MY_PATH, 'test/PNEUMONIA')
    print(dir)

    # 이미지 파일 경로
    P_path = []
    for f in os.listdir(dir):
        P_path.append(os.path.join(dir, f)) # dir과 파일 이름을 합침 -> 파일의 총 경로가 됨
    P_tot = len(P_path)
    print('평가용 폐렴 이미지 수 :', P_tot)

    # 총 평가용 이미지 수
    V_tot = N_tot + P_tot
    print('평가용 이미지 수 :', V_tot)

In [7]:
# 평가용 이미지 데이터 처리 (화상도 통일)
# 정상 라벨은 0, 폐렴은 1
def test_resize():
    global V_tot, N_tot, P_tot, N,path, P_path
    global X_test, Y_test

    print('정상 이미지 처리 시작')
    begin = time()

    # 초기화
    X_test = np.zeros((V_tot, MY_RES, MY_RES, 3)) # 데이터 수, 이미지 화소수, 칼라
    Y_test = np.zeros((V_tot,)) # 1차원 벡터

    for i, path in enumerate(N_path):    
        #tmp = load_img(path)
        #print(np.array(tmp).shape)
        #plt.imshow(tmp)
        #plt.show()

        # 이미지 크기 조정(축소)
        img = load_img(path,
                       target_size=(MY_RES, MY_RES))
        
        #plt.imshow(tmp)
        #plt.show()

        X_test[i] = img
        Y_test[i] = 0 # 라벨 정보
        #print(X_train[i].shape)

    end = time()
    print('정상 이미지 처리 소요 시간 : {:.2f}'.format(end - begin))

    print('폐렴 이미지 처리 시작')
    begin = time()

    for i, path in enumerate(P_path):
        img = load_img(path, 
                       target_size=(MY_RES, MY_RES))
        X_test[i + N_tot] = img 
        Y_test[i + N_tot] = 1 # 폐렴은 1

    end = time()
    print('폐렴 이미지 처리 소요 시간 : {:.2f}'.format(end - begin))

    # [0, 255] 사이의 8-비트 칼라 정보를 [0,1]로 스케일링(정규화)
    #print(X_test[0])
    X_test = X_test / 255.0

In [8]:
# 사분할 데이터 저장
def save_data():
    with open('chest-arrays.npy', 'wb') as f:
      np.save(f, X_train)
      np.save(f, Y_train)
      np.save(f, X_test)
      np.save(f, Y_test)

    print('사분할 데이터 파일 완성')

In [9]:
# 사분할 데이터 읽기
def read_data():
    global X_train, Y_train, X_test, Y_test

    with open('chest-arrays.npy', 'rb') as f:
        X_train = np.load(f)
        Y_train = np.load(f)
        X_test = np.load(f)
        Y_test = np.load(f)

    print('사분할 데이터 파일 읽기 완성')

In [10]:
# 데이터 처리 컨트롤 타워
if DATA_MODE:
    train_path()
    train_resize()
    test_path()
    test_resize()
    save_data()
else:
    read_data()

/content/drive/MyDrive/Colab Notebooks/data/chest/train/NORMAL
학습용 정상 이미지 수 : 500
/content/drive/MyDrive/Colab Notebooks/data/chest/train/PNEUMONIA
학습용 폐렴 이미지 수 : 500
학습용 이미지 수 : 1000
정상 이미지 처리 시작
정상 이미지 처리 소요 시간 : 0.38
폐렴 이미지 처리 시작
폐렴 이미지 처리 소요 시간 : 440.54
/content/drive/MyDrive/Colab Notebooks/data/chest/test/NORMAL
평가용 정상 이미지 수 : 100
/content/drive/MyDrive/Colab Notebooks/data/chest/test/PNEUMONIA
평가용 폐렴 이미지 수 : 100
평가용 이미지 수 : 200
정상 이미지 처리 시작
정상 이미지 처리 소요 시간 : 0.04
폐렴 이미지 처리 시작
폐렴 이미지 처리 소요 시간 : 83.03
사분할 데이터 파일 완성


In [11]:
# 데이터 모양 확인
print('학습용 입력 데이터 모양:', X_train.shape)
print('학습용 출력 데이터 모양:', Y_train.shape)

print('평가용 입력 데이터 모양:', X_test.shape)
print('평가용 출력 데이터 모양:', Y_test.shape)

학습용 입력 데이터 모양: (1000, 180, 180, 3)
학습용 출력 데이터 모양: (1000,)
평가용 입력 데이터 모양: (200, 180, 180, 3)
평가용 출력 데이터 모양: (200,)


3. 인공 신경망 구현

In [12]:
# 합성곱 블럭 함수
def conv_block(filters, inputs):
    # Conv2D 빠른 버전 
    x = SeparableConv2D(filters=filters,
                        kernel_size=3,
                        activation='relu',
                        padding='same')(inputs)

    x = SeparableConv2D(filters=filters,
                        kernel_size=3,
                        activation='relu',
                        padding='same')(x) # 전 단의 아웃풋이 현재 단의 인풋으로

    x = BatchNormalization()(x)

    output = MaxPool2D(pool_size=2)(x)

    return output

In [13]:
# Dense 블럭 함수
def dense_block(units, drop, inputs):
    x = Dense(units=units,
              activation='relu')(inputs)

    x = BatchNormalization()(x)

    output = Dropout(drop)(x)

    return output

In [14]:
# 전체 CNN 구현

# 입력층
my_input = Input(shape=(MY_SHAPE)) # 180*180*3

# 블럭 2
x = Conv2D(filters=16,
           kernel_size=3,
           activation='relu',
           padding='same')(my_input)

x = Conv2D(filters=16,
           kernel_size=3,
           activation='relu',
           padding='same')(x)

x = MaxPool2D(pool_size=2)(x) # 2배로 줄어듦

# 블럭 3
x = conv_block(32, x) # 전 단의 출력이 다음 단의 입력

# 블럭 4
x = conv_block(64, x)

# 블럭 5
x = conv_block(128, x)

# 블럭 6
x = conv_block(256, x)
x = Dropout(0.2)(x)

# 블럭 7
x = Flatten()(x)

# 블럭 8
x = dense_block(512, 0.7, x) 
x = dense_block(128, 0.5, x)
x = dense_block(64, 0.3, x)

# 블럭 9
my_output = Dense(units=1,
          activation='sigmoid')(x)

# 모델 만들기
model = Model(inputs=my_input,
              outputs=my_output)

print('CNN 요약')
model.summary()

CNN 요약
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 180, 180, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 180, 180, 16)      448       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 180, 180, 16)      2320      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 90, 90, 16)        0         
_________________________________________________________________
separable_conv2d (SeparableC (None, 90, 90, 32)        688       
_________________________________________________________________
separable_conv2d_1 (Separabl (None, 90, 90, 32)        1344      
_________________________________________________________________
batch_normalization (BatchNo (None, 90, 90, 32)       

4. 인공 신경망 학습

In [15]:
# CNN 학습
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc'])

# 학습 진행
begin = time()
print('학습 시작')

model.fit(x=X_train,
          y=Y_train,
          epochs=MY_EPOCH,
          batch_size=MY_BATCH,
          verbose=1)

end = time()
print('학습 시간 : {:.2f}초'.format(end-begin))

학습 시작
Epoch 1/200
10/10 [==============================] - 14s 492ms/step - loss: 0.3924 - acc: 0.8014
Epoch 2/200
10/10 [==============================] - 5s 496ms/step - loss: 0.0154 - acc: 0.9993
Epoch 3/200
10/10 [==============================] - 5s 487ms/step - loss: 0.0118 - acc: 1.0000
Epoch 4/200
10/10 [==============================] - 5s 498ms/step - loss: 0.0052 - acc: 1.0000
Epoch 5/200
10/10 [==============================] - 5s 497ms/step - loss: 0.0035 - acc: 1.0000
Epoch 6/200
10/10 [==============================] - 5s 490ms/step - loss: 0.0051 - acc: 1.0000
Epoch 7/200
10/10 [==============================] - 5s 496ms/step - loss: 0.0030 - acc: 1.0000
Epoch 8/200
10/10 [==============================] - 5s 484ms/step - loss: 0.0021 - acc: 1.0000
Epoch 9/200
10/10 [==============================] - 5s 501ms/step - loss: 0.0017 - acc: 1.0000
Epoch 10/200
10/10 [==============================] - 5s 497ms/step - loss: 0.0018 - acc: 1.0000
Epoch 11/200
10/10 [============

In [16]:
score = model.evaluate(X_test, Y_test, verbose=1)
print(score[0]) # 최종 손실
print(score[1]) # 최종 정확도

7/7 [==============================] - 1s 55ms/step - loss: 1.6889e-06 - acc: 1.0000
1.6888532172742998e-06
1.0
